In [1]:
from pydataset import data
import pandas as pd
from env import user, host, password
import numpy as np

def get_db_url(user, host, password, db_name):
    return f"mysql+pymysql://{user}:{password}@{host}/{db_name}"

### 1. Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:

In [ ]:
mpg = data("mpg")
mpg.columns

#### * On average, which manufacturer has the best miles per gallon?

In [ ]:
mpg.groupby("manufacturer")[['cty', 'hwy']].agg("mean").nlargest(1, ['cty', 'hwy'], keep = "all")

### * How many different manufacturers are there?

In [ ]:
print(f"There are {len(mpg.groupby('manufacturer'))} manufacturers.")

#### * How many different models are there?

In [ ]:
print(f"There are {len(mpg.groupby('model'))} models.")

#### * Do automatic or manual cars have better miles per gallon?

In [ ]:
def assign_transmission(trans):
    if trans.find("auto") > -1:
        return "auto"
    else:
        return "manual"

mpg['auto_or_manual'] = mpg.trans.apply(assign_transmission)
mpg.groupby('auto_or_manual')[['cty', 'hwy']].agg("mean")

### 2. Joining and Merging

In [ ]:
import numpy as np
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

In [ ]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

In [ ]:
# Right join would have all the roles whether or not there is a match
pd.merge(users, roles, left_on = "role_id", right_on = "id", how = "right")

In [ ]:
# Outer join would return results from both tables whether or not there is a match
pd.merge(users, roles, left_on = "role_id", right_on = "id", how = "outer")

In [ ]:
# If you drop the foreign keys you will not be able to join the tables
users.drop(columns = ['role_id'], inplace = True)
roles.drop(columns = ['id'], inplace = True)
pd.merge(users, roles, left_on = "role_id", right_on = "id", how = "inner")

### 3. Getting data from SQL databases

#### * Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url formatted like in the examples in this lesson.

In [2]:
def get_db_url(user, host, password, db_name):
    return f"mysql+pymysql://{user}:{password}@{host}/{db_name}"

#### * Use your function to obtain a connection to the employees database.

In [3]:
from env import user, host, password

In [ ]:
url = get_db_url(user, host, password, "employees")
query = """
    SELECT *
        FROM employees
            JOIN dept_emp USING(emp_no)
        WHERE to_date > now();
"""

pd.read_sql(query, url)

#### * Intentionally make a typo in the database url. What kind of error message do you see?

In [ ]:
pd.read_sql(query, url + "typo")

#### * Intentionally make an error in your SQL query. What does the error message look like?

In [ ]:
pd.read_sql(query + "typo", url)

#### * Read the employees and titles tables into two separate dataframes

In [ ]:
employees = pd.read_sql(query, url)
employees

In [ ]:
titles = pd.read_sql("SELECT * FROM titles;", url)
titles

#### * Visualize the number of employees with each title.

In [ ]:
titles.groupby('title').emp_no.agg("count")

In [ ]:
titles.groupby('title').emp_no.agg("count").plot.barh()

#### * Join the employees and titles dataframes together.

In [ ]:
emp_and_title = pd.merge(employees, titles, left_on = "emp_no", right_on = "emp_no")

#### * Visualize how frequently employees change titles.

In [ ]:
emp_and_title.groupby('emp_no').title.agg("count")

In [ ]:
emp_and_title.groupby('emp_no').title.agg("count").head(20).plot.bar(ylabel = "titles held")

#### * For each title, find the hire date of the employee that was hired most recently with that title.

In [ ]:
emp_and_title.groupby('title').hire_date.agg("max")

#### * Write the code necessary to create a cross tabulation of the number of titles by department.

In [ ]:
dept_query = """SELECT *
                    FROM departments;"""
departments = pd.read_sql(dept_query, url)
emp_title_dept = pd.merge(emp_and_title, departments, left_on = "dept_no", right_on = "dept_no")
emp_title_dept

In [ ]:
pd.crosstab(emp_title_dept.title, emp_title_dept.dept_name)

### 4. Use your get_db_url function to help you explore the data from the chipotle database. Use the data to answer the following questions:

In [ ]:
chipotle_url = get_db_url(user, host, password, "chipotle")
chipotle_query = """SELECT *
                        FROM orders"""

chipotle = pd.read_sql(chipotle_query, chipotle_url)
chipotle

#### * What is the total price for each order?

In [ ]:
def convert_to_float(price):
    price = price.replace("$", "")
    return float(price)

chipotle['float_price'] = chipotle.item_price.apply(convert_to_float)
chipotle.groupby('order_id').float_price.agg("sum")

#### * What are the most popular 3 items?

In [ ]:
chipotle.groupby('item_name').quantity.agg("count").sort_values(ascending = False).head(3)

#### * Which item has produced the most revenue?

In [ ]:
chipotle.groupby('item_name').float_price.agg("sum").sort_values(ascending = False).head(1)

### Bonus:  For even more practice with pandas, you can do the exercises from the SQL module, but instead of using SQL to do the aggregation, sorting, joining, etc, use pandas. That is, read the data from all of the tables into pandas dataframes and manipulate the dataframes.

In [5]:
world_url = get_db_url(user, host, password, "world")

In [4]:
world_query = """SELECT *
                    FROM city;"""

cities = pd.read_sql(world_query, world_url)
cities

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200
...,...,...,...,...,...
4074,4075,Khan Yunis,PSE,Khan Yunis,123175
4075,4076,Hebron,PSE,Hebron,119401
4076,4077,Jabaliya,PSE,North Gaza,113901
4077,4078,Nablus,PSE,Nablus,100231


#### * What languages are spoken in Santa Monica?

In [5]:
lang_query = """SELECT *
                    FROM countrylanguage;"""
languages = pd.read_sql(lang_query, world_url)
languages

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9
...,...,...,...,...
979,ZMB,Tongan,F,11.0
980,ZWE,English,T,2.2
981,ZWE,Ndebele,F,16.2
982,ZWE,Nyanja,F,2.2


In [ ]:
cities_languages = pd.merge(cities, languages, left_on = "CountryCode", right_on = "CountryCode")
language_count = cities_languages.groupby('Name').Language.agg("count")
language_count["Santa Monica"]
# 12 languages spoken in Santa Monica

#### * How many different countries are in each region?

In [6]:
country_query = """SELECT *
                        FROM country"""
countries = pd.read_sql(country_query, world_url) 
countries

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,YEM,Yemen,Asia,Middle East,527968.0,1918.0,18112000,59.8,6041.0,5729.0,Al-Yaman,Republic,Ali Abdallah Salih,1780.0,YE
235,YUG,Yugoslavia,Europe,Southern Europe,102173.0,1918.0,10640000,72.4,17000.0,NaN,Jugoslavija,Federal Republic,Vojislav Koštunica,1792.0,YU
236,ZAF,South Africa,Africa,Southern Africa,1221037.0,1910.0,40377000,51.1,116729.0,129092.0,South Africa,Republic,Thabo Mbeki,716.0,ZA
237,ZMB,Zambia,Africa,Eastern Africa,752618.0,1964.0,9169000,37.2,3377.0,3922.0,Zambia,Republic,Frederick Chiluba,3162.0,ZM


In [7]:
countries.groupby("Region").Name.agg("count")

Region
Antarctica                    5
Australia and New Zealand     5
Baltic Countries              3
British Islands               2
Caribbean                    24
Central Africa                9
Central America               8
Eastern Africa               20
Eastern Asia                  8
Eastern Europe               10
Melanesia                     5
Micronesia                    7
Micronesia/Caribbean          1
Middle East                  18
Nordic Countries              7
North America                 5
Northern Africa               7
Polynesia                    10
South America                14
Southeast Asia               11
Southern Africa               5
Southern Europe              15
Southern and Central Asia    14
Western Africa               17
Western Europe                9
Name: Name, dtype: int64

#### * What is the population for each region?

In [8]:
countries.groupby("Region").Population.agg("sum")

Region
Antarctica                            0
Australia and New Zealand      22753100
Baltic Countries                7561900
British Islands                63398500
Caribbean                      38140000
Central Africa                 95652000
Central America               135221000
Eastern Africa                246999000
Eastern Asia                 1507328000
Eastern Europe                307026000
Melanesia                       6472000
Micronesia                       543000
Micronesia/Caribbean                  0
Middle East                   188380700
Nordic Countries               24166400
North America                 309632000
Northern Africa               173266000
Polynesia                        633050
South America                 345780000
Southeast Asia                518541000
Southern Africa                46886000
Southern Europe               144674200
Southern and Central Asia    1490776000
Western Africa                221672000
Western Europe                183

#### * What is the population for each continent?

In [9]:
countries.groupby("Continent").Population.agg("sum")

Continent
Africa            784475000
Antarctica                0
Asia             3705025700
Europe            730074600
North America     482993000
Oceania            30401150
South America     345780000
Name: Population, dtype: int64

#### * What is the average life expectancy globally?

In [10]:
countries.LifeExpectancy.agg("mean")

66.48603603603601

#### * What is the average life expectancy for each region, each continent? Sort the results from shortest to longest

In [13]:
countries.groupby("Region").LifeExpectancy.agg("mean").sort_values().dropna()

Region
Southern Africa              44.820000
Central Africa               50.311111
Eastern Africa               50.810526
Western Africa               52.741176
Southern and Central Asia    61.350000
Southeast Asia               64.400000
Northern Africa              65.385714
Melanesia                    67.140000
Micronesia                   68.085714
Baltic Countries             69.000000
Eastern Europe               69.930000
Middle East                  70.566667
Polynesia                    70.733333
South America                70.946154
Central America              71.025000
Caribbean                    73.058333
Eastern Asia                 75.250000
North America                75.820000
Southern Europe              76.528571
British Islands              77.250000
Western Europe               78.255556
Nordic Countries             78.333333
Australia and New Zealand    78.800000
Name: LifeExpectancy, dtype: float64

In [15]:
countries.groupby("Continent").LifeExpectancy.agg("mean").sort_values().dropna()

Continent
Africa           52.571930
Asia             67.441176
Oceania          69.715000
South America    70.946154
North America    72.991892
Europe           75.147727
Name: LifeExpectancy, dtype: float64

#### * Find all the countries whose local name is different from the official name

In [23]:
different_name = countries[countries.Name != countries.LocalName]
different_name.loc[:, ['Name', 'LocalName']]

,Name,LocalName
1,Afghanistan,Afganistan/Afqanestan
4,Albania,Shqipëria
6,Netherlands Antilles,Nederlandse Antillen
7,United Arab Emirates,Al-Imarat al-´Arabiya al-Muttahida
9,Armenia,Hajastan
10,American Samoa,Amerika Samoa
11,Antarctica,–
12,French Southern territories,Terres australes françaises
15,Austria,Österreich
16,Azerbaijan,Azärbaycan


### Sakila Database

In [24]:
sakila_url = get_db_url(user, host, password, "sakila")
actor_query = """SELECT *
                    FROM actor;"""

#### * Display the first and last names in all lowercase of all the actors.

In [25]:
actors = pd.read_sql(actor_query, sakila_url)
actors

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33
...,...,...,...,...
195,196,BELA,WALKEN,2006-02-15 04:34:33
196,197,REESE,WEST,2006-02-15 04:34:33
197,198,MARY,KEITEL,2006-02-15 04:34:33
198,199,JULIA,FAWCETT,2006-02-15 04:34:33


In [32]:
actors['full_name'] = actors.first_name.str.lower() + " " + actors.last_name.str.lower()
actors

,actor_id,first_name,last_name,last_update,full_name
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33,penelope guiness
1,2,NICK,WAHLBERG,2006-02-15 04:34:33,nick wahlberg
2,3,ED,CHASE,2006-02-15 04:34:33,ed chase
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33,jennifer davis
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33,johnny lollobrigida
...,...,...,...,...,...
195,196,BELA,WALKEN,2006-02-15 04:34:33,bela walken
196,197,REESE,WEST,2006-02-15 04:34:33,reese west
197,198,MARY,KEITEL,2006-02-15 04:34:33,mary keitel
198,199,JULIA,FAWCETT,2006-02-15 04:34:33,julia fawcett


#### * You need to find the ID number, first name, and last name of an actor, of whom you know only the first name, "Joe." What is one query would you could use to obtain this information?

In [37]:
joes = actors[actors.first_name == "JOE"]
joes[['first_name', 'last_name', 'actor_id']]

,first_name,last_name,actor_id
8,JOE,SWANK,9


#### * Find all actors whose last name contain the letters "gen"

In [41]:
last_name_gen = actors[actors.last_name.str.find("GEN") > -1]
last_name_gen[['actor_id', 'last_name', 'first_name', 'full_name']]

,actor_id,last_name,first_name,full_name
13,14,BERGEN,VIVIEN,vivien bergen
40,41,DEGENERES,JODIE,jodie degeneres
106,107,DEGENERES,GINA,gina degeneres
165,166,DEGENERES,NICK,nick degeneres


#### * Find all actors whose last names contain the letters "li". This time, order the rows by last name and first name, in that order.

In [40]:
last_name_li = actors[actors.last_name.str.find("LI") > - 1]
last_name_li[['actor_id', 'last_name', 'first_name', 'full_name']].sort_values(['last_name', 'first_name'])

,actor_id,last_name,first_name,full_name
85,86,CHAPLIN,GREG,greg chaplin
81,82,JOLIE,WOODY,woody jolie
33,34,OLIVIER,AUDREY,audrey olivier
14,15,OLIVIER,CUBA,cuba olivier
171,172,WILLIAMS,GROUCHO,groucho williams
136,137,WILLIAMS,MORGAN,morgan williams
71,72,WILLIAMS,SEAN,sean williams
82,83,WILLIS,BEN,ben willis
95,96,WILLIS,GENE,gene willis
163,164,WILLIS,HUMPHREY,humphrey willis
